In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_rows=150
pd.options.display.max_columns=100
pd.options.display.max_colwidth=200
import datetime as dt
import warnings
warnings.filterwarnings(action='ignore')
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score,f1_score
from sklearn.model_selection import KFold,GroupKFold,ShuffleSplit,StratifiedKFold
import gc
import lightgbm as lgb
from tqdm import tqdm_notebook as tqdm
from imblearn.over_sampling import SMOTE
import mlcrate as mlc

In [ ]:
path = '/kaggle/input/wns-wizard/'

In [ ]:
#Reduce the memory usage - Inspired by Panchajanya Banerjee
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


In [ ]:
train_df = reduce_mem_usage(pd.read_csv(f'{path}train.csv',parse_dates=['impression_time',],dayfirst=True))
test_df = reduce_mem_usage(pd.read_csv(f'{path}test.csv',parse_dates=['impression_time',],dayfirst=True))
item_data = reduce_mem_usage(pd.read_csv(f'{path}item_data.csv'))
view_log = reduce_mem_usage(pd.read_csv(f'{path}view_log.csv',parse_dates=['server_time',],dayfirst=True))

In [ ]:
_ = view_log.groupby(['user_id'])['device_type'].unique().reset_index()
_['device_encode']=-1
for k,i in enumerate(tqdm( _['device_type'])):
    s = ''
    for ii in i:
        if ii=='android':
            s+='0'
        elif ii=='web':
            s='1'
        else:
            s+='2'
    s = ''.join(sorted(s,reverse=True))
    _.ix[k,'device_encode']=int(s)

_.drop(columns='device_type',inplace=True)
_.ix[_['device_encode']==10,'device_encode']=1
view_log=pd.merge(view_log,_,how='left',on='user_id')


In [ ]:
view_log['device_encode'].isnull().sum()

In [ ]:
from collections import Counter

def get_class_weights(y):
    counter = Counter(y)
    majority = max(counter.values())
    return  {cls: round(float(majority)/float(count), 2) for cls, count in counter.items()}

class_weights = get_class_weights(train_df['is_click'])
print(class_weights)

In [ ]:
#view_log['device_encode'] = view_log['device_encode'].astype(np.int8)

In [ ]:
train_df['is_click'].value_counts()

In [ ]:
train_df.shape

In [ ]:
## is_click==1
10862/237609

In [ ]:
train_df.head(100)

In [ ]:
test_df.head(100)

In [ ]:
view_log.head(100)

In [ ]:
train_df['is_train']=1
test_df['is_train']=0

In [ ]:
date_cols = ['impression_time']
for date_col in date_cols:
    train_df[date_col + "_in_seconds"] = (train_df[date_col] - dt.datetime(2000,1,1)).dt.total_seconds()
    test_df[date_col + "_in_seconds"] = (test_df[date_col] - dt.datetime(2000,1,1)).dt.total_seconds()
    
    train_df[date_col + "_month"] = train_df[date_col].dt.month
    test_df[date_col + "_month"] = test_df[date_col].dt.month
    
    train_df[date_col + "_hour"] = train_df[date_col].dt.hour
    test_df[date_col + "_hour"] = test_df[date_col].dt.hour
    
    #train_df[date_col + "_year"] = train_df[date_col].dt.year
    #test_df[date_col + "_year"] = test_df[date_col].dt.year
    
    train_df[date_col + "_week"] = train_df[date_col].dt.week
    test_df[date_col + "_week"] = test_df[date_col].dt.week
    
    train_df[date_col + "_weekDay"] = train_df[date_col].dt.weekday
    test_df[date_col + "_weekDay"] = test_df[date_col].dt.weekday

In [ ]:
train_df['is_click'] = train_df['is_click'].astype(np.int8)

In [ ]:
train_df.head()

In [ ]:
dist = pd.DataFrame(np.zeros((train_df['impression_time_hour'].nunique(),4)),columns=['Var','Shape','Zeros','Ones'])
for i,var in enumerate(train_df['impression_time_hour'].unique()):
    mask = train_df['impression_time_hour']==var
    size = train_df[mask].shape[0]
    mask0 = train_df['is_click']==0
    mask1 = train_df['is_click']==1
    ones = train_df[mask & mask1].shape[0]
    zeros = train_df[mask & mask0].shape[0]
    dist.iloc[i,:]=[var,int(size),int(zeros),int(ones)]
dist.sort_values(by='Ones',ascending=False)

In [ ]:
train_df[train_df['is_click']==1]

In [ ]:
print('Train min/max date: %s / %s' % (train_df.impression_time.min().date(), train_df.impression_time.max().date()))
print('Test  min/max date: %s / %s' % ( test_df.impression_time.min().date(),  test_df.impression_time.max().date()))
print('')
print('Number of days in train: %d' % ((train_df.impression_time.max() - train_df.impression_time.min()).days + 1))
print('Number of days in test:  %d' % (( test_df.impression_time.max() -  test_df.impression_time.min()).days + 1))
print('')
print('Train shape: %d rows' % train_df.shape[0])
print('Test shape: %d rows'  % test_df.shape[0])

## Train_Data Preprocessing

In [ ]:
train_test = pd.concat([train_df,test_df],sort=False,axis=0).reset_index(drop=True)
#del train_df,test_df
gc.collect()

In [ ]:
train_test = train_test.sort_values(by=['impression_time']).reset_index(drop=True)
train_test.head()

In [ ]:
cols = ['after_one_day_samples','prev_one_day_samples']
for col in cols:
    train_test[col]=-1

In [ ]:
for i in tqdm(range(train_test.shape[0])):
    s = train_test.iloc[i,[0,1,2]]
    current_time= s[1]

    mask = train_test['user_id']==s[2]
    mask_1 = train_test.iloc[i,0]
    
    
    
    s_next_day = pd.date_range(s[1], freq='1D', periods=2)
    after_one_day =s_next_day[1]
    s_prev_day = pd.date_range(end=s[1], freq='1D', periods=2)
    prev_one_day =s_prev_day[0]
        
        
    after_one_day_samples = train_test[mask & (train_test['impression_time'] >current_time ) & (train_test['impression_time'] <= after_one_day)].shape[0]
    prev_one_day_samples = train_test[mask & (train_test['impression_time'] >prev_one_day ) & (train_test['impression_time'] <= current_time)].shape[0]
    
    train_test.iloc[i,[-2,-1]]=after_one_day_samples,prev_one_day_samples
   

In [ ]:
cols = ['after_one_hour_samples','prev_one_hour_samples',]
for col in cols:
    train_test[col]=-1

In [ ]:
for i in tqdm(range(train_test.shape[0])):
    s = train_test.iloc[i,[0,1,2]]
    current_time= s[1]

    mask = train_test['user_id']==s[2]
    mask_1 = train_test.iloc[i,0]
    
    
    
    s_next_hour = pd.date_range(s[1], freq='1H', periods=2)
    #print(s[1],s_next_hour[1])
    after_one_hour =s_next_hour[1]
    s_prev_hour = pd.date_range(end=s[1], freq='1H', periods=2)
    #print(s_prev_hour[0],s[1])
    prev_one_hour =s_prev_hour[0]
        
        
        
    after_one_hour_samples = train_test[mask & (train_test['impression_time'] >current_time ) & (train_test['impression_time'] < after_one_hour)].shape[0]
    prev_one_hour_samples = train_test[mask & (train_test['impression_time'] >prev_one_hour ) & (train_test['impression_time'] < current_time)].shape[0]
    train_test.iloc[i,[-2,-1]]=after_one_hour_samples,prev_one_hour_samples
   

In [ ]:
cols = ['after_one_month_samples','prev_one_month_samples']
for col in cols:
    train_test[col]=-1

In [ ]:
for i in tqdm(range(train_df.shape[0])):
    s = train_test.iloc[i,[0,1,2]]
    current_time= s[1]

    mask = train_test['user_id']==s[2]
    mask_1 = train_test.iloc[i,0]
    
    
    
    s_next_month = pd.date_range(s[1], freq='1M', periods=2)
    after_one_month =s_next_month[1]
    s_prev_month = pd.date_range(end=s[1], freq='1M', periods=2)
    prev_one_month =s_prev_month[0]
        
        
    after_one_month_samples = train_test[mask & (train_test['impression_time'] >current_time ) & (train_test['impression_time'] < after_one_month)].shape[0]
    prev_one_month_samples = train_test[mask & (train_test['impression_time'] >prev_one_month ) & (train_test['impression_time'] < current_time)].shape[0]
    #print(after_one_month_samples,prev_one_month_samples)
    train_test.iloc[i,[-2,-1]]=after_one_month_samples,prev_one_month_samples
   

In [ ]:
cols = ['after_all_samples','prev_all_samples']
for col in cols:
    train_test[col]=-1

In [ ]:
for i in tqdm(range(train_df.shape[0])):
    s = train_test.iloc[i,[0,1,2]]
    current_time= s[1]

    mask = train_test['user_id']==s[2]
    mask_1 = train_test.iloc[i,0]
    prev_all_samples = train_test[mask & (train_test['impression_time'] <current_time)].shape[0]
    after_all_samples = train_test[mask & (train_test['impression_time'] >current_time)].shape[0]
        #print(after_one_hour_samples,prev_one_hour_samples,after_one_day_samples,prev_one_day_samples,after_one_week_samples,prev_one_week_samples,\
              #after_one_month_samples,prev_one_month_samples,prev_all_samples,after_all_samples)
    train_test.iloc[i,[-2,-1]]=after_all_samples,prev_all_samples
    
   

In [ ]:
cols = ['after_one_week_samples','prev_one_week_samples']
for col in cols:
    train_test[col]=-1
    

In [ ]:
for i in tqdm(range(train_df.shape[0])):
    s = train_test.iloc[i,[0,1,2]]
    current_time= s[1]

    mask = train_test['user_id']==s[2]
    mask_1 = train_test.iloc[i,0]
    
    
    
    s_next_week = pd.date_range(s[1], freq='1W', periods=2)
    after_one_week =s_next_week[1]
    s_prev_week = pd.date_range(end=s[1], freq='1W', periods=2)
    prev_one_week =s_prev_week[0]
        
        
    after_one_week_samples = train_test[mask & (train_test['impression_time'] >current_time ) & (train_test['impression_time'] < after_one_week)].shape[0]
    prev_one_week_samples = train_test[mask & (train_test['impression_time'] >prev_one_week ) & (train_test['impression_time'] < current_time)].shape[0]
        
    #print(after_one_week_samples,prev_one_week_samples)
    train_test.iloc[i,[-2,-1]]=after_one_week_samples,prev_one_week_samples
   

In [ ]:
train_test.columns

In [ ]:
train_test.drop(columns=['after_one_hour_samples','after_one_day_samples','after_one_week_samples','after_one_month_samples','after_all_samples'],inplace=True)

In [ ]:
train_test.shape

In [ ]:
def hour_encode(val):
    if val in list(range(2,8)):
        return 0
    elif val in list(range(9,14)):
        return 1
    else:
        return 2

In [ ]:
#train_test['hour_encode'] = train_test['impression_time_hour'].apply(hour_encode)

In [ ]:
train_df.replace(np.inf,np.nan,inplace=True)
test_df.replace(np.inf,np.nan,inplace=True)
train_df =  train_df.replace(-np.inf, np.nan)
test_df = test_df.replace(-np.inf, np.nan)

In [ ]:
train_app_unique = train_df['app_code'].unique() 
test_app_unique = test_df['app_code'].unique()
s = [i for i in train_app_unique if i not in test_app_unique]
ss = [i for i in test_app_unique if i not in train_app_unique]

In [ ]:
##unique app_code
train_test.ix[train_test['app_code'].isin(ss+s),'app_code_prsent']=1
train_test.ix[~train_test['app_code'].isin(ss+s),'app_code_prsent']=0
#train_test['app_code_prsent'] = train_test['app_code_prsent'].astype(np.int8)

In [ ]:
## impression_time
_ = train_test.groupby('user_id')['impression_time_in_seconds'].mean().reset_index()
_.columns = ['user_id','user_impression_time_in_second_mean']
train_test = pd.merge(train_test,_,how='left',on='user_id')
train_test["user_cumnum_impression_time"] = train_test.groupby("user_id")["impression_time_in_seconds"].cumcount().values

In [ ]:
_ = train_test.groupby('user_id').size().reset_index()
_.columns = ['user_id','user_id_count']
train_test = pd.merge(train_test,_,how='left',on='user_id')

In [ ]:
train_test['prev_impression_time'] = train_test.groupby('user_id')['impression_time_in_seconds'].shift(1)
train_test['time_gap_between_prev'] = train_test['impression_time_in_seconds']-train_test['prev_impression_time']
train_test['next_impression_time'] = train_test.groupby('user_id')['impression_time_in_seconds'].shift(-1)
train_test['time_gap_between_next'] = train_test['impression_time_in_seconds']-train_test['next_impression_time']
train_test.drop(columns=['prev_impression_time','next_impression_time'],inplace=True)

In [ ]:
train_test['prev_impression_time_2'] = train_test.groupby('user_id')['impression_time_in_seconds'].shift(2)
train_test['time_gap_between_prev_2'] = train_test['impression_time_in_seconds']-train_test['prev_impression_time_2']
train_test['next_impression_time_2'] = train_test.groupby('user_id')['impression_time_in_seconds'].shift(-2)
train_test['time_gap_between_next_2'] = train_test['impression_time_in_seconds']-train_test['next_impression_time_2']
train_test.drop(columns=['prev_impression_time_2','next_impression_time_2'],inplace=True)

In [ ]:
train_test['prev_impression_time_3'] = train_test.groupby('user_id')['impression_time_in_seconds'].shift(3)
train_test['time_gap_between_prev_3'] = train_test['impression_time_in_seconds']-train_test['prev_impression_time_3']
train_test['next_impression_time_3'] = train_test.groupby('user_id')['impression_time_in_seconds'].shift(-3)
train_test['time_gap_between_next_3'] = train_test['impression_time_in_seconds']-train_test['next_impression_time_3']
train_test.drop(columns=['prev_impression_time_3','next_impression_time_3'],inplace=True)

In [ ]:
_ = train_test.groupby(['user_id'])['app_code'].nunique().reset_index()
_.columns=['user_id','userId_appCode_unique']
train_test = pd.merge(train_test,_,how='left',on='user_id')
_ = train_test.groupby(['user_id','app_code']).size().reset_index()
_.columns=['user_id','app_code','userId_appCode_size']
train_test = pd.merge(train_test,_,how='left',on=['user_id','app_code'])
del _
gc.collect()


In [ ]:
## os_version
_ = train_test.groupby(['user_id'])['os_version'].nunique().reset_index()
_.columns=['user_id','userId_osVersion_unique']
train_test = pd.merge(train_test,_,how='left',on='user_id')
_ = train_test.groupby(['user_id','os_version']).size().reset_index()
_.columns=['user_id','os_version','userId_osVersion_size']
train_test = pd.merge(train_test,_,how='left',on=['user_id','os_version'])
del _
gc.collect()

In [ ]:
## is_4G
_ = train_test.groupby(['user_id'])['is_4G'].nunique().reset_index()
_.columns=['user_id','userId_is4G_unique']
train_test = pd.merge(train_test,_,how='left',on='user_id')
_ = train_test.groupby(['user_id','is_4G']).size().reset_index()
_.columns=['user_id','is_4G','userId_is4G_size']
train_test = pd.merge(train_test,_,how='left',on=['user_id','is_4G'])
del _
gc.collect()

In [ ]:
mask  = train_test['userId_appCode_size']==train_test['user_id_count']
train_test.ix[mask,'userId_appCode_size']=-1
mask1 = train_test['userId_osVersion_size']==train_test['user_id_count']
train_test.ix[mask1,'userId_osVersion_size']=-1
mask2 = train_test['userId_is4G_size']==train_test['user_id_count']
train_test.ix[mask2,'userId_is4G_size']=-1

In [ ]:
## impression_id
_ = train_test.groupby(['user_id'])['impression_id'].nunique().reset_index()
_.columns=['user_id','user_impression_nunique']
train_test = pd.merge(train_test,_,how='left',on=['user_id'])
train_test['user_impression_ratio'] = train_test['user_impression_nunique']/train_test['user_id_count']

In [ ]:
train_test.shape

In [ ]:
view_log = pd.merge(view_log,item_data,on='item_id',how='left')
del item_data
gc.collect()

In [ ]:
view_log.head(100)

In [ ]:

def label_encode_df(dataframe,cols):
    for col in cols:
        le=preprocessing.LabelEncoder()
        dataframe[str(col)+'_encode']=le.fit_transform(dataframe[col].astype(str))

In [ ]:
label_encode_df(train_test,['os_version'])
label_encode_df(view_log,['device_type'])

In [ ]:
date_cols = ['server_time']
for date_col in date_cols:
    view_log[date_col + "_in_seconds"] = (view_log[date_col] - dt.datetime(2000,1,1)).dt.total_seconds()
    view_log[date_col + "_month"] = view_log[date_col].dt.month
    #train_df[date_col + "_year"] = train_df[date_col].dt.year
    #test_df[date_col + "_year"] = test_df[date_col].dt.year
    view_log[date_col + "_week"] = view_log[date_col].dt.week
    view_log[date_col + "_weekDay"] = view_log[date_col].dt.weekday

android = 0<br>
andoid+iphone=20<br>
android+web=1<br>
iphone=2<br>

In [ ]:
#view_log['device_encode'].unique()

In [ ]:
_ = view_log.groupby('user_id')['device_type'].nunique().reset_index()
_.columns= ['user_id','device_type_nunique']
view_log = pd.merge(view_log,_,on='user_id',how='left')
train_test = pd.merge(train_test,_,on='user_id',how='left')


_ = view_log.groupby('user_id')['device_encode'].first().reset_index()
train_test = pd.merge(train_test,_,on='user_id',how='left')
del _
gc.collect()

In [ ]:
train_test['device_encode'].unique()

In [ ]:
user_view = view_log['user_id'].unique()
user_train = train_test['user_id'].unique()
len([i for i in user_train if i not in user_view ])

In [ ]:
view_log[view_log['user_id']==19223]

In [ ]:
_ = view_log.groupby('user_id')['session_id'].nunique().reset_index()
_.columns= ['user_id','session_id_nunique']
view_log = pd.merge(view_log,_,on='user_id',how='left')
train_test = pd.merge(train_test,_,on='user_id',how='left')
del _
gc.collect()


In [ ]:
_ = view_log.groupby('user_id')['item_id'].nunique().reset_index()
_.columns= ['user_id','item_id_nunique']
view_log = pd.merge(view_log,_,on='user_id',how='left')
train_test = pd.merge(train_test,_,on='user_id',how='left')
del _
gc.collect()

In [ ]:
_ = view_log.groupby(['user_id']).size().reset_index()
_.columns = ['user_id','no_of_time_user_connected']
view_log = pd.merge(view_log,_,on='user_id',how='left')
train_test = pd.merge(train_test,_,on='user_id',how='left')
del _ 
gc.collect()

In [ ]:
_ = view_log.groupby('user_id')['server_time_in_seconds'].mean().reset_index()
_.columns = ['user_id','user_server_time_in_second_mean']
view_log = pd.merge(view_log,_,on='user_id',how='left')
train_test = pd.merge(train_test,_,on='user_id',how='left')
del _ 
gc.collect()

In [ ]:
train_test['user_session_ratio'] = train_test['session_id_nunique']/train_test['no_of_time_user_connected']
train_test['user_item_ratio'] = train_test['item_id_nunique']/train_test['no_of_time_user_connected']

view_log['user_session_ratio'] = view_log['session_id_nunique']/view_log['no_of_time_user_connected']
view_log['user_item_ratio'] = view_log['item_id_nunique']/view_log['no_of_time_user_connected']

In [ ]:
_ = view_log.groupby(['user_id','session_id'])['server_time_in_seconds'].first().reset_index()
_.columns = ['user_id','session_id','first']
view_log = pd.merge(view_log,_,how='left',on=['user_id','session_id'])

_ = view_log.groupby(['user_id','session_id'])['server_time_in_seconds'].last().reset_index()
_.columns = ['user_id','session_id','last']
view_log = pd.merge(view_log,_,how='left',on=['user_id','session_id'])
view_log['overall_session_time'] = view_log['last']-view_log['first']
del _ 
gc.collect()


_ = view_log.groupby(['user_id'])['overall_session_time'].sum().reset_index()
_.columns = ['user_id','overall_time_sum']
view_log = pd.merge(view_log,_,how='left',on='user_id')
train_test = pd.merge(train_test,_,on='user_id',how='left')

del _
gc.collect()

In [ ]:
view_log.drop(columns=['first','last','overall_session_time'],inplace=True)

In [ ]:
_ = view_log.groupby(["user_id"])["item_price"].agg(["min", "max", "mean", "std"]).reset_index()
_.columns   = ['user_id','user_item_price_min','user_item_price_max','user_item_price_mean','user_item_price_std']
train_test = pd.merge(train_test,_,on='user_id',how='left')
view_log = pd.merge(view_log,_,how='left',on='user_id')
del _
gc.collect()

In [ ]:
train_test.head()

In [ ]:
#train_test['diff_user_impression_session_time_in_second_mean'] = train_test['user_impression_time_in_second_mean']-train_test['user_server_time_in_second_mean']

In [ ]:
view_log.head(100)

In [ ]:
train_test.head()

In [ ]:
train_test['user_app_price_rank'] = train_test.groupby(['user_id','app_code'])['user_item_price_mean'].rank()

In [ ]:
train_test.columns

In [ ]:
train_test.shape

In [ ]:
_ = pd.pivot_table(train_test, index="user_id", columns="impression_time_weekDay", values="impression_id", aggfunc="count", fill_value=0).reset_index()
train_test = pd.merge(train_test, _, on="user_id", how="left")

_ = pd.pivot_table(train_test, index="user_id", columns="impression_time_month", values="impression_id", aggfunc="count", fill_value=0).reset_index()
train_test = pd.merge(train_test, _, on="user_id", how="left")

_ = pd.pivot_table(train_test, index="user_id", columns="impression_time_hour", values="impression_id", aggfunc="count", fill_value=0).reset_index()
train_test = pd.merge(train_test, _, on="user_id", how="left")

_ = pd.pivot_table(train_test, index="user_id", columns="impression_time_week", values="impression_id", aggfunc="count", fill_value=0).reset_index()
train_test = pd.merge(train_test, _, on="user_id", how="left")


#_ = pd.pivot_table(train_test, index="user_id", columns="app_code", values="impression_id", aggfunc="count", fill_value=0).reset_index()
#train_test = pd.merge(train_test, _, on="user_id", how="left")

del _
gc.collect()


In [ ]:
_ = train_test.groupby(['user_id'])['app_code'].shift(1)
train_test['prev_app_code_diff'] = (train_test['app_code'] == _).astype(int)
_ = train_test.groupby(['user_id'])['app_code'].shift(-1)
train_test['next_app_code_diff'] = (train_test['app_code'] == _).astype(int)


In [ ]:
#train_test['device_encode'].fillna(3,inplace=True)
#train_test['device_encode'] = train_test['device_encode'].astype(np.int8)

In [ ]:
train_test.head()

In [ ]:
train_test.shape

In [ ]:
train_test = train_test.sort_values(by=['impression_time','user_id'])

In [ ]:
train_test['user_impressionTime_diff_in_seconds']  = train_test.groupby(['user_id'])['impression_time_in_seconds'].diff()

In [ ]:
gc.collect()

In [ ]:
train_test.shape

In [ ]:
train_test = train_test.sort_values(by=['impression_time']).reset_index(drop=True)
gc.collect()
train_test.head()

In [ ]:
import mlcrate as mlc
import pickle as pkl

from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Dense, Flatten, Concatenate, Dot, Reshape, Add, Subtract
from keras import objectives
from keras import backend as K
from keras import regularizers 
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2

In [ ]:
import numpy as np
import tensorflow as tf
import random as rn

# The below is necessary in Python 3.2.3 onwards to
# have reproducible behavior for certain hash-based operations.
# See these references for further details:
# https://docs.python.org/3.4/using/cmdline.html#envvar-PYTHONHASHSEED
# https://github.com/fchollet/keras/issues/2280#issuecomment-306959926

import os

def init_seeds(seed):
    os.environ['PYTHONHASHSEED'] = '0'

    # The below is necessary for starting Numpy generated random numbers
    # in a well-defined initial state.

    np.random.seed(seed)

    # The below is necessary for starting core Python generated random numbers
    # in a well-defined state.

    rn.seed(seed)

    # Force TensorFlow to use single thread.
    # Multiple threads are a potential source of
    # non-reproducible results.
    # For further details, see: https://stackoverflow.com/questions/42022950/which-seeds-have-to-be-set-where-to-realize-100-reproducibility-of-training-res

    session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

    from keras import backend as K

    # The below tf.set_random_seed() will make random number generation
    # in the TensorFlow backend have a well-defined initial state.
    # For further details, see: https://www.tensorflow.org/api_docs/python/tf/set_random_seed

    tf.set_random_seed(seed)

    sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
    K.set_session(sess)
    return sess

In [ ]:
train_test['device_encode'].unique()

In [ ]:
train_test['device_encode'].max()

In [ ]:
features = ['app_code', 'os_version_encode', 'is_4G','device_encode']
f_size  = [int(train_test[f].max()) + 1 for f in features]
f_size

In [ ]:
X = train_test.groupby(features)['impression_id'].count()
X = X.unstack().fillna(0)
X = X.stack().astype('float32')
X = np.log1p(X).reset_index()
X.columns=features + ['num']
X.head()

In [ ]:
X_train = [X[f].values for f in features]
y_train = (X[['num']].values).astype('float32')

In [ ]:
(X.num > 0).mean()

In [ ]:
w_train = ((y_train > 0).astype('float32') + 1).ravel()

In [ ]:
k_latent = 2
embedding_reg = 0.0002
kernel_reg = 0.1

def get_embed(x_input, x_size, k_latent):
    if x_size > 0: #category
        embed = Embedding(x_size, k_latent, input_length=1, 
                          embeddings_regularizer=l2(embedding_reg))(x_input)
        embed = Flatten()(embed)
    else:
        embed = Dense(k_latent, kernel_regularizer=l2(embedding_reg))(x_input)
    return embed

def build_model_1(X, f_size):
    dim_input = len(f_size)
    
    input_x = [Input(shape=(1,)) for i in range(dim_input)] 
     
    biases = [get_embed(x, size, 1) for (x, size) in zip(input_x, f_size)]
    
    factors = [get_embed(x, size, k_latent) for (x, size) in zip(input_x, f_size)]
    
    s = Add()(factors)
    
    diffs = [Subtract()([s, x]) for x in factors]
    
    dots = [Dot(axes=1)([d, x]) for d,x in zip(diffs, factors)]
    
    x = Concatenate()(biases + dots)
    x = BatchNormalization()(x)
    output = Dense(1, activation='relu', kernel_regularizer=l2(kernel_reg))(x)
    model = Model(inputs=input_x, outputs=[output])
    opt = Adam(clipnorm=0.5)
    model.compile(optimizer=opt, loss='mean_squared_error')
    output_f = factors + biases
    model_features = Model(inputs=input_x, outputs=output_f)
    return model, model_features

In [ ]:

model, model_features = build_model_1(X_train, f_size)
model.summary()

In [ ]:
model_features.summary()

In [ ]:
n_epochs = 30

try:
    del sess
except:
    pass
sess = init_seeds(0)

batch_size = 8
print(batch_size)
model, model_features = build_model_1(X_train, f_size)
earlystopper = EarlyStopping(patience=3, verbose=1)

model.fit(X_train,  y_train, 
          epochs=n_epochs, batch_size=batch_size, verbose=1, shuffle=True, 
          validation_data=(X_train, y_train), 
          sample_weight=w_train,
          callbacks=[earlystopper],
         )

In [ ]:
X_pred = model_features.predict(X_train, batch_size=batch_size)

In [ ]:
factors = X_pred[:len(features)]

biases = X_pred[len(features):2*len(features)]

for f, X_p in zip(features, factors):
    for i in range(k_latent):
        X['%s_fm_factor_%d' % (f, i)] = X_p[:,i]

for f, X_p in zip(features, biases):
    X['%s_fm_bias' % (f)] = X_p[:,0]

X.head()

In [ ]:
train_test = pd.merge(train_test,X,on=['app_code','os_version_encode','is_4G','device_encode'],how='left')

In [ ]:
train_test.shape

In [ ]:
#train_test.drop(columns=['num'],inplace=True)

In [ ]:
train_test.to_pickle('train_test_2.pkl')  

In [ ]:
train_test.isnull().sum()

In [ ]:
train_test.info()

In [ ]:
list(train_test.columns)

In [ ]:
train_test.shape

In [ ]:
cols_to_exclude = ["impression_time", "is_click",'is_train','impression_id','os_version','device_type','diff_user_impression_session_time_in_second_mean']
cols_to_use = [col for col in train_test.columns if col not in cols_to_exclude]

In [ ]:
train_test.fillna(-99,inplace=True)

In [ ]:
train_df = train_test[train_test['is_train']==1]
test_df = train_test[train_test['is_train']==0]

In [ ]:
train_df.shape

In [ ]:
del train_test

In [ ]:
gc.collect()

In [ ]:
train_df['is_click'] = train_df['is_click'].astype(np.int8)

In [ ]:
train_df[train_df['is_click']==1].shape

In [ ]:
lgb_params = {
                    'objective':'binary',
                    'boosting_type':'gbdt',
                    'metric':'auc',
                    'n_jobs':-1,
                    'learning_rate':0.001,
                    'num_leaves': 2**8,
                    'max_depth':-1,
                    'tree_learner':'serial',
                    'colsample_bytree': 0.7,
                    'subsample_freq':1,
                    'subsample':0.7,
                    'max_bin':255,
                    'verbose':-1,
                    'seed': 42,
                } 

In [ ]:
params = {}
params["objective"] = "binary"
params["max_depth"] = -1
params["num_leaves"] = 30
params["min_data_in_leaf"] = 50
#  params["min_sum_hessian_in_leaf"] = 50
params["learning_rate"] = 0.01
params["bagging_fraction"] = 0.8
params["feature_fraction"] = 0.2
params["feature_fraction_seed"] = 42
params["bagging_freq"] = 1
params["bagging_seed"] = 42
params["lambda_l2"] = 3
params["lambda_l1"] = 3
params["verbosity"] = -1
params['metric']='auc'

In [ ]:
target = train_df['is_click']
folds = StratifiedKFold(n_splits=5,shuffle=True,random_state=2019)
oof = np.zeros(len(train_df))
predictions = np.zeros(len(test_df))
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(tqdm(folds.split(train_df.values, target.values,train_df['user_id']))):
    print("Fold {}".format(fold_))
    trn_data = lgb.Dataset(train_df.iloc[trn_idx][cols_to_use], label=target.iloc[trn_idx])
    val_data = lgb.Dataset(train_df.iloc[val_idx][cols_to_use], label=target.iloc[val_idx])
    num_round = 400000
    clf = lgb.train(lgb_params, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=500, early_stopping_rounds = 200)
    oof[val_idx] = clf.predict(train_df.iloc[val_idx][cols_to_use], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = cols_to_use
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(test_df[cols_to_use], num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(roc_auc_score(target, oof)))

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

auc = roc_auc_score(train_df['is_click'], oof)
fpr, tpr, thresholds = roc_curve(train_df['is_click'], oof)
plt.figure()
plt.plot(fpr, tpr, label='Adaptive Boosting Classifier (area = %0.2f)' % auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate'), plt.ylabel('True Positive Rate'), plt.title('Receiver operating characteristic')
plt.legend(loc="lower right")
# plt.savefig('Log_ROC')
plt.show()

In [ ]:
cols = (feature_importance_df[["Feature", "importance"]]
        .groupby("Feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:150].index)
best_features = feature_importance_df.loc[feature_importance_df.Feature.isin(cols)]

plt.figure(figsize=(14,28))
sns.barplot(x="importance", y="Feature", data=best_features.sort_values(by="importance",ascending=False))
plt.title('Features importance (averaged/folds)')
plt.tight_layout()
plt.savefig('FI.png')

In [ ]:
sub_df = pd.DataFrame(test_df[["impression_id"]])
sub_df['is_click'] = predictions 
sub_df.to_csv("lgbm-0.7522605358233.csv", index=False)